# Continuous Optimisation HW1

In [ ]:
# Imports
import numpy as np
import scipy
import matplotlib as plt

# Question 1
Implement phi(x, P), bigphi(X, P), f(X, P, y).

In [ ]:
# Question 1
#TODO Vectorise all the functions


def h(x: np.ndarray) -> float:
    """
    Gaussian filter

    :x: np.ndarray[1x2]
    """
    return np.e**(-np.inner(x, x)/0.01)  # Always take sigma = 0.1


def phi(x, P):
    """
    Calculate contribution of each 'true' star to observed image
    
    :x: np.ndarray[1 x 2]
    :P: np.ndarray[1 x n^2]
    """
    phi = []
    for i in range(np.size(P)):
        phi.append(h(P[i]-x))
    return phi


def bigphi(X, P):
    """
    Calculate image observed, based on K-star positions X
    
    :X: np.ndarray[2 x K]
    :P: np.ndarray[1 x n^2]
    """
    K = X.shape[1]  # Number of stars
    bigphi = []
    for i in range(K):
        xc  = np.array([X[0][i], X[1][i]])
        bigphi += phi(xc, P)
    return bigphi


def  f(X, P, y):
    """
    Calculate squared error of estimate bigphi(X)
    
    :X: np.ndarray[2 x K]
    :P: np.ndarray[1 x n^2]
    :y: np.ndarray[1 x n^2]
    """
    sizeP = int(np.size(P))
    return (1/(2*sizeP)) * np.linalg.norm(bigphi(X, P)-y)**2




TypeError: 'builtin_function_or_method' object is not subscriptable

# Question 2

In [13]:
true_positions = np.array([[0], [0]])
positions = np.array([[0.2, 0.15], [-0.2, 0.15], [-0.2, -0.15], [0.2, -0.15]]).T
y = np.array([0.2, 0.2, 0.2, 0.2]).T
n = 2
print(positions)
print(f(np.array([[0.2], [0.2]]), positions, y))

[[ 0.2  -0.2  -0.2   0.2 ]
 [ 0.15  0.15 -0.15 -0.15]]


ValueError: operands could not be broadcast together with shapes (4,) (2,) 

# Question 3